This file creates a graph with the actual prices overlapped with another dataset

In [1]:
# Library imports
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
# Defines all utility functions
def load_data(file_name: str, datetime_col: str, timezone: str) -> pd.DataFrame:
    DATA_PATH = Path.cwd().parent / 'data'
    df = pd.read_csv(DATA_PATH / file_name)

    # Read the datetime and convert it to UTC timezone if necessary
    df[datetime_col] = pd.to_datetime(df[datetime_col])
    df['datetime_utc'] = (
        df[datetime_col]
        .dt.tz_localize(timezone, ambiguous='NaT', nonexistent='shift_forward')
        .dt.tz_convert('UTC')
    )
    
    return df

def reduce_data(df: pd.DataFrame, num_entries_per_day: int) -> pd.DataFrame:
    # Filter the dataset to only a certain number of entries per day
    hours_between_selected = 24 / num_entries_per_day
    df_reduced = df[(df['hour'] % hours_between_selected == 0) & (df['minute'] == 0)]
    return df_reduced

def merge_data_sources(df1: pd.DataFrame, df2: pd.DataFrame, merge_col: str = 'datetime_utc') -> pd.DataFrame:
    return df1.merge(df2, on='datetime_utc', how='inner')

def plot_single_series(df: pd.DataFrame, x_col: str, y_col: str, x_label: str = None, y_label: str = None, graph_title: str = None):
    if not x_label:
        x_label = x_col
    if not y_label:
        y_label = y_col
    
    df_without_na = df.dropna(subset=[x_col, y_col])
    x = df_without_na[x_col]
    y = df_without_na[y_col]
    
    plt.plot(x, y)
    plt.xlabel(x_label)
    plt.xticks(rotation=45)
    plt.ylabel(y_label)
    if graph_title: plt.title(graph_title)
    plt.show()

def plot_two_y_series(df: pd.DataFrame, x_col: str, y1_col: str, y2_col: str, 
                       x_label: str = None, y1_label: str = None, y2_label: str = None, graph_title: str = None):
    if not x_label:
        x_label = x_col
    if not y1_label:
        y1_label = y1_col
    if not y2_label:
        y2_label = y2_col
    
    df_without_na = df.dropna(subset=[x_col, y1_col, y2_col])
    x = df_without_na[x_col]
    y1 = df_without_na[y1_col]
    y2 = df_without_na[y2_col]
    
    plt.plot(x, y1, label=y1_label)
    plt.plot(x, y2, label=y2_label)
    plt.xlabel(x_label)
    plt.xticks(rotation=45)
    plt.ylabel('y axis')
    plt.legend()
    if graph_title: plt.title(graph_title)
    plt.show()

In [10]:
# Load the actual data
df_actual_price = load_data('imbalance_actual.csv', datetime_col='datetime_utc', timezone='UTC')
print(f'Loaded actual data with shape {df_actual_price.shape}', end='\n\n')

# Load the forecast data
df_forecasted_price = load_data('imbalance_forecast.csv', datetime_col='datetime_utc', timezone='UTC')
print(f'Loaded forecast data with shape {df_forecasted_price.shape}', end='\n\n')

# Load the weather data
df_weather_quarter_hourly = load_data('weather_data-quarter_hourly.csv', datetime_col='time', timezone='Europe/Berlin')
print(f'Loaded quarter-hourly weather data with shape {df_weather_quarter_hourly.shape}', end='\n\n')
df_weather_hourly = load_data('weather_data-hourly.csv', datetime_col='time', timezone='Europe/Berlin')
print(f'Loaded hourly weather data with shape {df_weather_hourly.shape}')

# Load 
# TODO - complete this

Loaded actual data with shape (49559, 5)

Loaded forecast data with shape (105949, 6)

Loaded quarter-hourly weather data with shape (50400, 14)

Loaded hourly weather data with shape (12600, 4)


In [7]:
df_actual_price

,datetime_utc,date,hour,minute,price_eur_mwh
0,2024-05-21 22:00:00+00:00,2024-05-21,22,0,106.39
1,2024-05-21 22:15:00+00:00,2024-05-21,22,15,149.28
2,2024-05-21 22:30:00+00:00,2024-05-21,22,30,151.17
3,2024-05-21 22:45:00+00:00,2024-05-21,22,45,226.60
4,2024-05-21 23:00:00+00:00,2024-05-21,23,0,204.76
...,...,...,...,...,...
49554,2025-10-20 02:30:00+00:00,2025-10-20,2,30,-56.90
49555,2025-10-20 02:45:00+00:00,2025-10-20,2,45,94.02
49556,2025-10-20 03:00:00+00:00,2025-10-20,3,0,103.56
49557,2025-10-20 03:15:00+00:00,2025-10-20,3,15,93.99


In [8]:
df_forecasted_price

,datetime_utc,date,hour,minute,second,price_eur_mwh
0,2025-07-24 13:52:00+00:00,2025-07-24,13,52,0,100.0
1,2025-07-24 13:53:00+00:00,2025-07-24,13,53,0,100.0
2,2025-07-24 13:54:00+00:00,2025-07-24,13,54,0,100.0
3,2025-07-24 13:55:00+00:00,2025-07-24,13,55,0,100.0
4,2025-07-24 13:56:00+00:00,2025-07-24,13,56,0,100.0
...,...,...,...,...,...,...
105944,2025-10-20 16:32:00+00:00,2025-10-20,16,32,0,76.0
105945,2025-10-20 16:33:00+00:00,2025-10-20,16,33,0,76.0
105946,2025-10-20 16:34:00+00:00,2025-10-20,16,34,0,76.0
105947,2025-10-20 16:35:00+00:00,2025-10-20,16,35,0,76.0


In [ ]:
# Reduce the data to less entries for plotting
start_shape = df_actual_price.shape
df_actual_price = reduce_data(df_actual_price, num_entries_per_day=1)
print(f'Reduced actual data from shape {start_shape} to shape {df_actual_price.shape}')

start_shape = df_forecasted_price.shape
df_forecasted_price = reduce_data(df_forecasted_price, num_entries_per_day=1)
print(f'Reduced forecast data from shape {start_shape} to shape {df_forecasted_price.shape}')

start_shape = df_weather_quarter_hourly.shape
df_actual_price = reduce_data(df_weather_quarter_hourly, num_entries_per_day=1)
print(f'Reduced quarter-hourly weather data from shape {start_shape} to shape {df_weather_quarter_hourly.shape}')

start_shape = df_weather_hourly.shape
df_forecasted_price = reduce_data(df_weather_hourly, num_entries_per_day=1)
print(f'Reduced hourly weather data from shape {start_shape} to shape {df_weather_hourly.shape}')